PYTHON-12. Работа с файлами разных форматов

# START

In [ ]:
import warnings; warnings.simplefilter('ignore')

import random

import pandas as pd
import numpy as np
import os
import re

import json  
from pprint import pprint  

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


%matplotlib inline


In [ ]:
project_dir = r'/content/drive/My Drive/Colab Notebooks/data/u3/'
os.listdir(project_dir)

['log.csv',
 'users.csv',
 'data_flats.csv',
 'latlons.csv',
 'train.csv',
 'recipes.json']

## 12.1 Формат JSON

In [ ]:
with open(project_dir + 'recipes.json') as f:
   recipes = json.load(f)

In [ ]:
pprint(recipes[0:3])

[{'cuisine': 'greek',
  'id': 10259,
  'ingredients': ['romaine lettuce',
                  'black olives',
                  'grape tomatoes',
                  'garlic',
                  'pepper',
                  'purple onion',
                  'seasoning',
                  'garbanzo beans',
                  'feta cheese crumbles']},
 {'cuisine': 'southern_us',
  'id': 25693,
  'ingredients': ['plain flour',
                  'ground pepper',
                  'salt',
                  'tomatoes',
                  'ground black pepper',
                  'thyme',
                  'eggs',
                  'green tomatoes',
                  'yellow corn meal',
                  'milk',
                  'vegetable oil']},
 {'cuisine': 'filipino',
  'id': 20130,
  'ingredients': ['eggs',
                  'pepper',
                  'salt',
                  'mayonaise',
                  'cooking oil',
                  'green chilies',
                  'grilled chicken bre

In [ ]:
len(recipes), type(recipes)

(500, list)

In [ ]:
recipes[0]['id']

10259

In [ ]:
recipes[0]['ingredients']

['romaine lettuce',
 'black olives',
 'grape tomatoes',
 'garlic',
 'pepper',
 'purple onion',
 'seasoning',
 'garbanzo beans',
 'feta cheese crumbles']

In [ ]:
find_id = 13121

for recipe in recipes:
    if recipe['id'] == find_id:
        print(recipe['cuisine'])
        break

thai


In [ ]:
# Task 12.1.1

recipes[len(recipes) - 1]['cuisine']

'british'

In [ ]:
# Task 12.1.2
len(recipes[len(recipes) - 1]['ingredients'])


17

In [ ]:
# Task 12.1.3
find_id = 17636
for recipe in recipes:
    if recipe['id'] == find_id:
        print(recipe['ingredients'])
        break



['tomato sauce', 'shredded carrots', 'spinach', 'part-skim mozzarella cheese', 'italian seasoning', 'english muffins, split and toasted', 'chopped onion', 'vegetable oil cooking spray', 'chopped green bell pepper']


In [ ]:
# Task 12.1.4
find_id = 42013
for recipe in recipes:
    if recipe['id'] == find_id:
        print(len(recipe['ingredients']))
        break


14


In [ ]:
# Task 12.1.5
find_id = 23629
for recipe in recipes:
    if recipe['id'] == find_id:
        print(recipe['ingredients'])
        break

['eggs', 'russet potatoes', 'mexican chorizo', 'black beans', 'salsa', 'canola oil', 'colby cheese', 'black olives', 'sour cream', 'avocado', 'half & half', 'goat cheese']


# END